<a href="https://colab.research.google.com/github/kridtapon/KlingerForce/blob/main/KlingerForce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.8 MB/s eta 0:00:00


In [22]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Klinger Oscillator
def calculate_klinger(data, long_period=34, short_period=55):
    """
    Calculate Klinger Oscillator.
    """
    # Price oscillator (difference between closing price and previous closing price)
    price_oscillator = 2 * (data['Close'] - data['Close'].shift(1)) / (data['High'] - data['Low'])

    # Volume oscillator (difference between volume and previous volume)
    volume_oscillator = 2 * (data['Volume'] - data['Volume'].shift(1)) / (data['Volume'] + data['Volume'].shift(1))

    # Klinger Oscillator
    klinger = price_oscillator * volume_oscillator
    klinger = klinger.rolling(window=long_period).sum() - klinger.rolling(window=short_period).sum()

    return klinger

# Function to calculate Ultimate Oscillator
def calculate_ultimate_oscillator(data, short_period=7, mid_period=14, long_period=28):
    """
    Calculate Ultimate Oscillator.
    """
    # Calculate the True Range
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift(1))
    low_close = np.abs(data['Low'] - data['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate Buying Pressure
    buying_pressure = data['Close'] - data['Low']

    # Calculate the Ultimate Oscillator
    bp_7 = buying_pressure.rolling(window=short_period).sum()
    tr_7 = true_range.rolling(window=short_period).sum()

    bp_14 = buying_pressure.rolling(window=mid_period).sum()
    tr_14 = true_range.rolling(window=mid_period).sum()

    bp_28 = buying_pressure.rolling(window=long_period).sum()
    tr_28 = true_range.rolling(window=long_period).sum()

    ultimate_oscillator = 100 * ((4 * bp_7 / tr_7) + (2 * bp_14 / tr_14) + (bp_28 / tr_28)) / 7

    return ultimate_oscillator

# Define the stock symbol and time period
symbol = 'SYF'  # SYF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Klinger Oscillator and Ultimate Oscillator
df['Klinger'] = calculate_klinger(df,long_period=30, short_period=65)
df['Ultimate_Oscillator'] = calculate_ultimate_oscillator(df,short_period=6, mid_period=12, long_period=28)

# Define Entry and Exit signals based on Klinger and Ultimate Oscillator
df['Entry'] = (df['Klinger'] > 0) & (df['Ultimate_Oscillator'] > 50)
df['Exit'] = (df['Klinger'] < 0) & (df['Ultimate_Oscillator'] < 50)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           510736.996971
Total Return [%]                       410.736997
Benchmark Return [%]                   104.995867
Max Gross Exposure [%]                      100.0
Total Fees Paid                       6920.098188
Max Drawdown [%]                        29.493551
Max Drawdown Duration                       268.0
Total Trades                                   15
Total Closed Trades                            15
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            73.333333
Best Trade [%]                          53.957322
Worst Trade [%]                        -20.290025
Avg Winning Trade [%]                   20.152342
Avg Losing Trade [%]                    -6.038466


In [21]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Function to calculate Klinger Oscillator
def calculate_klinger(data, long_period=34, short_period=55):
    price_oscillator = 2 * (data['Close'] - data['Close'].shift(1)) / (data['High'] - data['Low'])
    volume_oscillator = 2 * (data['Volume'] - data['Volume'].shift(1)) / (data['Volume'] + data['Volume'].shift(1))
    klinger = price_oscillator * volume_oscillator
    klinger = klinger.rolling(window=long_period).sum() - klinger.rolling(window=short_period).sum()
    return klinger

# Function to calculate Ultimate Oscillator
def calculate_ultimate_oscillator(data, short_period=7, mid_period=14, long_period=28):
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift(1))
    low_close = np.abs(data['Low'] - data['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    buying_pressure = data['Close'] - data['Low']

    bp_7 = buying_pressure.rolling(window=short_period).sum()
    tr_7 = true_range.rolling(window=short_period).sum()

    bp_14 = buying_pressure.rolling(window=mid_period).sum()
    tr_14 = true_range.rolling(window=mid_period).sum()

    bp_28 = buying_pressure.rolling(window=long_period).sum()
    tr_28 = true_range.rolling(window=long_period).sum()

    ultimate_oscillator = 100 * ((4 * bp_7 / tr_7) + (2 * bp_14 / tr_14) + (bp_28 / tr_28)) / 7

    return ultimate_oscillator

# Define the stock symbol and time period
symbol = 'SYF'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define parameter ranges for optimization
klinger_long_periods = range(30, 51, 5)
klinger_short_periods = range(50, 101, 5)
ultimate_short_periods = range(5, 11)
ultimate_mid_periods = range(10, 21, 2)
ultimate_long_periods = range(20, 31, 2)

# Function to run the backtest with different parameters
def optimize_parameters(klinger_long_periods, klinger_short_periods, ultimate_short_periods, ultimate_mid_periods, ultimate_long_periods):
    best_return = -np.inf
    best_params = None

    for klinger_long, klinger_short, ult_short, ult_mid, ult_long in product(klinger_long_periods, klinger_short_periods, ultimate_short_periods, ultimate_mid_periods, ultimate_long_periods):
        # Calculate Klinger Oscillator and Ultimate Oscillator
        df['Klinger'] = calculate_klinger(df, long_period=klinger_long, short_period=klinger_short)
        df['Ultimate_Oscillator'] = calculate_ultimate_oscillator(df, short_period=ult_short, mid_period=ult_mid, long_period=ult_long)

        # Define Entry and Exit signals based on Klinger and Ultimate Oscillator
        df['Entry'] = (df['Klinger'] > 0) & (df['Ultimate_Oscillator'] > 50)
        df['Exit'] = (df['Klinger'] < 0) & (df['Ultimate_Oscillator'] < 50)

        # Filter data for the test period (2020-2025)
        df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

        # Backtest using vectorbt
        portfolio = vbt.Portfolio.from_signals(
            close=df_filtered['Close'],
            entries=df_filtered['Entry'],
            exits=df_filtered['Exit'],
            init_cash=100_000,
            fees=0.001
        )

        # Calculate the Return of the portfolio
        total_return = portfolio.total_return()

        # Update best parameters if current Return is better
        if total_return > best_return:
            best_return = total_return
            best_params = (klinger_long, klinger_short, ult_short, ult_mid, ult_long)

    return best_params, best_return

# Run optimization
best_params, best_return = optimize_parameters(klinger_long_periods, klinger_short_periods, ultimate_short_periods, ultimate_mid_periods, ultimate_long_periods)

# Print the best parameters and Return
print(f"Best Parameters: Klinger Long: {best_params[0]}, Klinger Short: {best_params[1]}, Ultimate Short: {best_params[2]}, Ultimate Mid: {best_params[3]}, Ultimate Long: {best_params[4]}")
print(f"Best Return: {best_return}")


[*********************100%***********************]  1 of 1 completed


Best Parameters: Klinger Long: 30, Klinger Short: 65, Ultimate Short: 6, Ultimate Mid: 12, Ultimate Long: 28
Best Return: 4.107369969712435


In [26]:
# Define the stock symbol and time period
symbol = 'SYF'
start_date = '2020-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,204995.879476
Total Return [%],104.995879
Benchmark Return [%],104.995879
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],64.621645
